# Ensemble Learning

## Initial Imports

In [173]:
import warnings
warnings.filterwarnings('ignore')

In [174]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import calendar
from sklearn.preprocessing import LabelEncoder

In [175]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [176]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [177]:
#Instanciate the Label Encoder to encode the data for analysis
le = LabelEncoder()

#Encode the data using the le label encoder
le.fit(df["home_ownership"])
df["home_ownership"] = le.transform(df["home_ownership"])

le.fit(df["pymnt_plan"])
df["pymnt_plan"] = le.transform(df["pymnt_plan"])

#Use get_dummies to encode the rest of the columns
df = pd.get_dummies(df, columns=["debt_settlement_flag"])
df = pd.get_dummies(df, columns=["hardship_flag"])
df = pd.get_dummies(df, columns=["verification_status"])
df = pd.get_dummies(df, columns=["initial_list_status"])
df = pd.get_dummies(df, columns=["next_pymnt_d"])
df = pd.get_dummies(df, columns=["application_type"])
df.head()



,loan_amnt,int_rate,installment,home_ownership,annual_inc,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,...,hardship_flag_N,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,3,66000.0,Mar-2019,low_risk,0,27.24,0.0,...,1,0,1,0,0,1,0,1,1,0
1,25000.0,0.2000,929.09,1,105000.0,Mar-2019,low_risk,0,20.23,0.0,...,1,0,0,1,0,1,0,1,1,0
2,20000.0,0.2000,529.88,1,56000.0,Mar-2019,low_risk,0,24.26,0.0,...,1,0,0,1,0,1,0,1,1,0
3,10000.0,0.1640,353.55,3,92000.0,Mar-2019,low_risk,0,31.44,0.0,...,1,0,0,1,0,1,0,1,1,0
4,22000.0,0.1474,520.39,1,52000.0,Mar-2019,low_risk,0,18.76,0.0,...,1,1,0,0,0,1,0,1,1,0


In [178]:
# Print the column names of the dataframe
print(list(df))

['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc', 'issue_d', 'loan_status', 'pymnt_plan', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_

In [179]:
# Encode the date column
# First build a dictionary of months and their numbers

months_num = {
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12,
}

# Separate the month from the year using .split, then iterate through the rows
df['issue_d'] = df['issue_d'].str.split('-')
name_to_num = {name: num for num, name in enumerate(calendar.month_name) if num}
df.head()



,loan_amnt,int_rate,installment,home_ownership,annual_inc,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,...,hardship_flag_N,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,3,66000.0,"[Mar, 2019]",low_risk,0,27.24,0.0,...,1,0,1,0,0,1,0,1,1,0
1,25000.0,0.2000,929.09,1,105000.0,"[Mar, 2019]",low_risk,0,20.23,0.0,...,1,0,0,1,0,1,0,1,1,0
2,20000.0,0.2000,529.88,1,56000.0,"[Mar, 2019]",low_risk,0,24.26,0.0,...,1,0,0,1,0,1,0,1,1,0
3,10000.0,0.1640,353.55,3,92000.0,"[Mar, 2019]",low_risk,0,31.44,0.0,...,1,0,0,1,0,1,0,1,1,0
4,22000.0,0.1474,520.39,1,52000.0,"[Mar, 2019]",low_risk,0,18.76,0.0,...,1,1,0,0,0,1,0,1,1,0


In [180]:
# Create new columns using the separated values
df["issue_d_year"] = df["issue_d"].str[1]
df["issue_d_months_num"] = df["issue_d"].str[0].apply(lambda x: months_num[x])
df.drop(columns= "issue_d", axis=1, inplace= True)
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,verification_status_Source Verified,verification_status_Verified,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,issue_d_year,issue_d_months_num
0,10500.0,0.1719,375.35,3,66000.0,low_risk,0,27.24,0.0,0.0,...,1,0,0,1,0,1,1,0,2019,3
1,25000.0,0.2000,929.09,1,105000.0,low_risk,0,20.23,0.0,0.0,...,0,1,0,1,0,1,1,0,2019,3
2,20000.0,0.2000,529.88,1,56000.0,low_risk,0,24.26,0.0,0.0,...,0,1,0,1,0,1,1,0,2019,3
3,10000.0,0.1640,353.55,3,92000.0,low_risk,0,31.44,0.0,1.0,...,0,1,0,1,0,1,1,0,2019,3
4,22000.0,0.1474,520.39,1,52000.0,low_risk,0,18.76,0.0,1.0,...,0,0,0,1,0,1,1,0,2019,3


In [198]:
# Create a new csv file to send encoded dataframe for use
# file_path = Path("Resources/LoanStats_2019Q1_encoded.csv")
# df.to_csv(file_path, index=False)

In [211]:
# re-load the encoded dataframe from the new csv file
file_path = Path('Resources/LoanStats_2019Q1_encoded.csv')
loanstats_df = pd.read_csv(file_path)

## Split the Data into Training and Testing

In [212]:
# Create our features
X = loanstats_df.drop(columns= "loan_status")

# Create our target
y = loanstats_df["loan_status"]



In [213]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,...,verification_status_Source Verified,verification_status_Verified,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,issue_d_year,issue_d_months_num
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.0,21.778153,0.217766,0.497697,12.587340,...,0.373992,0.148001,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,2019.0,1.726172
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.0,20.199244,0.718367,0.758122,6.022869,...,0.483865,0.355104,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.743862
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.0,0.000000,0.000000,0.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019.0,1.000000
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.0,13.890000,0.000000,0.000000,8.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,2019.0,1.000000
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,0.0,19.760000,0.000000,0.000000,11.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,2019.0,2.000000
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,0.0,26.660000,0.000000,1.000000,16.000000,...,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,2019.0,2.000000
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,0.0,999.000000,18.000000,5.000000,72.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2019.0,3.000000


In [214]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [215]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 2)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [216]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [217]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [218]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [222]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf= BalancedRandomForestClassifier(n_estimators=100, random_state=2)

# Fit the training data to the random forest classifier
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=2)

In [236]:
# Calculated the balanced accuracy score
y_pred_rf = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred_rf)

0.7400475399482641

In [227]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_rf)

array([[   47,    34],
       [ 1715, 15409]], dtype=int64)

In [228]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.58      0.90      0.05      0.72      0.51        81
   low_risk       1.00      0.90      0.58      0.95      0.72      0.54     17124

avg / total       0.99      0.90      0.58      0.94      0.72      0.54     17205



In [233]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
importances_sorted = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.08099031683824008, 'total_rec_prncp'),
 (0.07325866371249525, 'total_rec_int'),
 (0.07258402616331029, 'last_pymnt_amnt'),
 (0.04860532808789486, 'total_pymnt'),
 (0.04639675299202408, 'total_pymnt_inv'),
 (0.026186213196595524, 'int_rate'),
 (0.025501902357964065, 'issue_d_months_num'),
 (0.01905150636740463, 'mths_since_recent_inq'),
 (0.018052194405217895, 'dti'),
 (0.017324212898481007, 'bc_open_to_buy'),
 (0.01701161349521556, 'max_bal_bc'),
 (0.016989467928796725, 'revol_bal'),
 (0.016481618823003856, 'installment'),
 (0.015649237174661004, 'mo_sin_old_rev_tl_op'),
 (0.015078109917845686, 'avg_cur_bal'),
 (0.014867851792151785, 'mo_sin_old_il_acct'),
 (0.014814637839443487, 'annual_inc'),
 (0.01476835934114023, 'total_bal_il'),
 (0.014724792532454027, 'out_prncp_inv'),
 (0.01451789660167252, 'bc_util'),
 (0.014367917406347347, 'il_util'),
 (0.013734394950283337, 'tot_hi_cred_lim'),
 (0.013710985140861097, 'out_prncp'),
 (0.013611008024900291, 'mths_since_rcnt_il'),
 (0.012979

### Easy Ensemble Classifier

In [234]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

enc = EasyEnsembleClassifier(n_estimators= 100, random_state= 2)
enc.fit(X_train, y_train)


EasyEnsembleClassifier(n_estimators=100, random_state=2)

In [235]:
# Calculated the balanced accuracy score
y_pred_enc = enc.predict(X_test)
balanced_accuracy_score(y_test, y_pred_enc)

0.9419809321117427

In [237]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_rf)

array([[   47,    34],
       [ 1715, 15409]], dtype=int64)

In [238]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_enc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.94      0.95      0.14      0.94      0.89        81
   low_risk       1.00      0.95      0.94      0.97      0.94      0.89     17124

avg / total       1.00      0.95      0.94      0.97      0.94      0.89     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Easy Ensemble Classifier

2. Which model had the best recall score?

    Easy Ensemble Classifier

3. Which model had the best geometric mean score?

    Easy Ensemble Classifier

4. What are the top three features?

   The top 3 features are: 'total_rec_prncp', 'total_rec_int',
 and 'last_pymnt_amnt'